In [2]:
import numpy as np

In [60]:
len(set(sample))

29

In [46]:
sample = ['jingle', 'grinch', 'carol', 'polar', 'visit', 'cheer', 'unwrap', 'stocking',
            'workshop', 'sing', 'the', 'holly', 'relax', 'ornament', 'chimney', 'magi',
            'naughty', 'is', 'eat', 'nutcracker', 'cheer', 'yuletide', 'and', 'holiday',
            'gifts', 'decorations', 'beard', 'sleigh', 'of', 'nice']

verbs = ["sing", "eat", "visit", "relax", "unwrap", "and", "of", "the", "is"]

vrbs_area = [len(verbs)//(2**(2-i)) for i in range(3)]
othrs_area = [(len(sample) - len(verbs))//(2**(2-i)) for i in range(3)]

mutation_chances = 1
dupl = True

In [88]:
def mutation_structure(p) :
    origin = np.array(p)
    lnth = len(origin)
    
    for _ in range(mutation_chances) :
        ## mutation rate : default 50%
        if np.random.random() > 0.33 :
            dice = np.random.randint(0, 4) ## choice mutation method
            
            ## swap
            if dice == 0 :
                swap_area = np.random.choice([k for k in range(lnth)], size = 2, replace = False)
                origin[swap_area[0]], origin[swap_area[1]] = origin[swap_area[1]], origin[swap_area[0]]
                
            ## move
            elif dice == 1 :
                moving_indx = np.random.randint(0, lnth)
                mover = origin[moving_indx]
                
                moving_area = np.random.randint(0, lnth)
                
                ## trick
                tmp = list(origin)
                del tmp[moving_indx]
                tmp.insert(moving_area, mover)
                
                origin = np.array(tmp)
                
            ## inverse
            elif dice == 2 :
                if np.random.random() < 0.5 :
                    width = np.random.randint(3, 5)
                    start = np.random.randint(0, lnth-width)
                    origin[start:start+width] = origin[start:start+width][::-1]
                
            ## scramble
            elif dice == 3 :
                if np.random.random() < 0.5 :
                    swap_size = np.random.randint(3, 5)
                    swap_area = np.random.choice([i for i in range(lnth)], size = swap_size, replace = False)
                    
                    origin[swap_area] = np.random.permutation(np.array(origin)[swap_area])
    
    return origin


def dupl_cleaner(p, clean_type) :
    """
    for handling duplication

    Args:
        p (List[str]): token list
        clean_type (str): if input "labeling" then add numbering for duplicated token

    Returns:
        prnt (List[str])
    """
    prnt = p.copy()
    
    if clean_type == "labeling" :    
        for t in set(p) :
            times = sum(np.array(p) == t)
            rep = 1
            
            if times > 1 :
                for j, k in enumerate(p) :
                    if k == t :
                        prnt[j] = f"{t}{rep}"
                        rep += 1
                        
    else :
        for j, t in enumerate(p) :
            try :
                int(t[-1])
                prnt[j] = t[:-1]
            except :
                pass
                    
    return prnt
    

def PMX(p1, p2, mapping_area) :
    parents = [p1, p2]
    childs = []
    
    for i in range(2) :
        main_set = parents[i]
        sub_set = parents[1-i]
        
        ## PMX
        if np.random.random() < 0.4 :
            width = np.random.randint(mapping_area[0], mapping_area[2])
        
        else :
            width = np.random.randint(mapping_area[0], mapping_area[1])
        
        start = np.random.randint(0, mapping_area[2] - width)
        child = [" " for _ in range(len(p1))]
        child[start:start+width] = main_set[start:start+width]
        
        ## mapping
        mapping_set = set(sub_set[start:start+width]) - set(main_set[start:start+width])
        
        for t in mapping_set :
            sub = np.where(np.array(sub_set) == t)[0][0]
            
            for k in range(mapping_area[2]) :
                sub = np.where(np.array(sub_set) == main_set[sub])[0][0]
                
                if (sub < start) | (sub >= start+width) :
                    child[sub] = t
                    break
            
        ## remain set
        current_set = [t for t in child if t != " "]
        remain_set = [t for t in sub_set if t not in current_set]
        
        empty_indx = [i for i, t in enumerate(child) if t == " "]
        
        for k, ind in enumerate(empty_indx) :
            child[ind] = remain_set[k]
            
        childs.append(child)
    
    return childs

def crossover(p1, p2, p3, verbs, mutation_chances = 1) :
    structure = [None for _ in range(len(p1))]
    vrbs = [[t for t in p2 if t in verbs], [t for t in p3 if t in verbs]]
    othrs = [[t for t in p2 if t not in verbs], [t for t in p3 if t not in verbs]]
    
    childs = []
    
    for i, t in enumerate(p1) :
        structure[i] = t in verbs
    
    if mutation_chances > 0 :
        structure = mutation_structure(structure)

    ## crossover
    if dupl :
        for i in range(2) :
            vrbs[i] = dupl_cleaner(vrbs[i], "labeling")
            othrs[i] = dupl_cleaner(othrs[i], "labeling")
    
    vrbs_childs = PMX(vrbs[0], vrbs[1], vrbs_area) ## two sets
    othrs_childs = PMX(othrs[0], othrs[1], othrs_area) ## two sets
    
    if dupl :
        for i in range(2) :
            vrbs_childs[i] = dupl_cleaner(vrbs_childs[i], "cleaning")
            othrs_childs[i] = dupl_cleaner(othrs_childs[i], "cleaning")
    
    ## merge childs
    for i in range(4) :
        child = ["" for _ in range(len(p1))]
        
        a = 0
        b = 0
        
        for j, s in enumerate(structure) :
            vrbs = vrbs_childs[i//2]
            othrs = othrs_childs[i%2]
            
            if s :
                child[j] = vrbs[a]
                a += 1
            else :
                child[j] = othrs[b]
                b += 1
            
        childs.append(child)
    
    return childs ## return 4 childs

In [77]:
## duplication 처리 함수 문제 없음
for _ in range(10000) :
    if set(dupl_cleaner(dupl_cleaner(sample, "labeling"), "cleaning")) != set(sample) :
        print("asdf")

In [49]:
p1 = np.random.permutation(sample)
p2 = np.random.permutation(sample)
p3 = np.random.permutation(sample)

`-` PMX에서 오류 발견

In [89]:
PMX(dupl_cleaner(p1, "labeling"), dupl_cleaner(p2, "labeling"), [7, 15, 30])

[[np.str_('visit'),
  np.str_('holiday'),
  np.str_('cheer1'),
  np.str_('jingle'),
  np.str_('the'),
  np.str_('chimney'),
  np.str_('beard'),
  np.str_('ornament'),
  np.str_('carol'),
  np.str_('magi'),
  np.str_('nice'),
  np.str_('eat'),
  np.str_('of'),
  np.str_('decorations'),
  np.str_('nutcracker'),
  np.str_('gifts'),
  np.str_('yuletide'),
  np.str_('unwrap'),
  np.str_('holly'),
  np.str_('grinch'),
  np.str_('workshop'),
  np.str_('sing'),
  np.str_('polar'),
  np.str_('and'),
  np.str_('cheer2'),
  np.str_('is'),
  np.str_('sleigh'),
  np.str_('stocking'),
  np.str_('naughty'),
  np.str_('relax')],
 [np.str_('nutcracker'),
  np.str_('holiday'),
  np.str_('unwrap'),
  np.str_('relax'),
  np.str_('gifts'),
  np.str_('visit'),
  np.str_('beard'),
  np.str_('ornament'),
  np.str_('carol'),
  np.str_('magi'),
  np.str_('nice'),
  np.str_('eat'),
  np.str_('of'),
  np.str_('holly'),
  np.str_('the'),
  np.str_('decorations'),
  np.str_('chimney'),
  np.str_('cheer1'),
  np.str

In [85]:
[" ".join(child) for child in PMX(p1, p2, [7, 15, 30])]

['visit eat and jingle magi beard chimney of is nutcracker holiday eat of is the decorations polar cheer yuletide workshop grinch and yuletide ornament nice gifts the carol cheer unwrap',
 'nutcracker holiday sing naughty relax sleigh visit is beard ornament magi nice eat carol of the carol unwrap holly grinch workshop sing polar and cheer is decorations yuletide chimney gifts']

In [51]:
childs = crossover(p1, p2, p3, verbs = verbs)

In [61]:
[len(set(child)) for child in childs]

[24, 25, 25, 26]

In [17]:
a = np.array([i for i in range(10)])
b = np.array([5,8,1,4,2,7,9,0,3,6])

PMX(a, b, [2, 5, 10])

[[np.int64(5),
  np.int64(8),
  np.int64(1),
  np.int64(2),
  np.int64(4),
  np.int64(5),
  np.int64(6),
  np.int64(7),
  np.int64(4),
  np.int64(6)],
 [np.int64(7),
  np.int64(0),
  np.int64(1),
  np.int64(4),
  np.int64(2),
  np.int64(7),
  np.int64(3),
  np.int64(5),
  np.int64(1),
  np.int64(6)]]